In [ ]:
!pip install peft -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 31.0 MB/s eta 0:00:00


In [ ]:
!pip install transformers[torch] -q

In [ ]:
import os
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, get_linear_schedule_with_warmup, TrainingArguments, Trainer
from torch.utils.data import Dataset, DataLoader, random_split
from peft import get_peft_model, PeftConfig, PeftModel, LoraConfig, get_peft_model_state_dict, prepare_model_for_int8_training, set_peft_model_state_dict
from transformers import AdamW, get_scheduler,TrainerCallback
from sklearn.model_selection import train_test_split

import torch.nn as nn

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from typing import List
from transformers import DataCollatorForSeq2Seq

In [ ]:
max_length = 512

In [ ]:
base_model = 'gpt2-medium'

In [ ]:
batch_size = 6

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Загрузка данных из CSV файла
df = pd.read_csv('/content/drive/MyDrive/МФТИ/nlp2/chatbot/Data/Barney2.csv')
df.head(5)

,name,text,context
0,Barney,"(on the phone) hey, so you know how I've alwa...",What was I doing? Your Uncle Marshall was tak...
1,Barney,"Okay, meet me at the bar in fifteen minutes, ...","Hey, you wanna do something tonight? Hey."
2,Barney,Where's your suit!? Just once when I say suit...,Hey. I did that one time.
3,Barney,It was a blazer!,"I did that one time. You know, ever since co..."
4,Barney,I see what this is about. Have you forgotten ...,"You know, ever since college it's been Marsha..."


In [ ]:
# Инициализация токенизатора
tokenizer = GPT2Tokenizer.from_pretrained(base_model)
tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

In [ ]:
# Создание класса датасета для обучения модели. В качестве лейбла передается следующее слово персонажа, которое модель должна предсказать

class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.data = dataframe.dropna()  # Удаление строк с NaN значениями

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = str(self.data.iloc[idx]['text']) + str(self.data.iloc[idx]['context'])
        encoding = self.tokenizer(text, max_length=self.max_length, truncation=True, padding='max_length', return_tensors='pt')
        input_ids = encoding.input_ids.squeeze(0)
        labels = input_ids.clone()  # Создаем копию input_ids
        labels[:-1] = input_ids[1:]  # Сдвигаем на один токен вперед
        labels[-1] = self.tokenizer.eos_token_id  # Заменяем последний токен на токен окончания последовательности
        return {"input_ids": input_ids, "labels": labels}


In [ ]:
dataset = CustomDataset(df, tokenizer, max_length)

In [ ]:
dataset[0]

{'input_ids': tensor([  357,   261,   262,  3072,     8, 17207,    11,   523,   345,   760,
           703,   314,  1053,  1464,   550,   257,  1517,   329,  2063,    12,
         43224,  4813,    30,  3894,    11,   783,   314,  1053,  1392,   257,
           649,  4004,    25, 28076,  4813,     0, 28076,  4813,   389,   262,
           649,  2063,    12,  1722,  1547,    13,  1867,   373,   314,  1804,
            30,  3406, 23169, 13606,   373,  2263,   262,  4094,  2239,   286,
           465,  1204,    11,   290,   502,    12,    40,  1101,  4585,   534,
         23169,    11, 41921,    13,   220, 14690,    11,   345, 18869,   466,
          1223,  9975,    30, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50

In [ ]:
def resume_training(model, resume_from_checkpoint):
    if resume_from_checkpoint:
        # Проверяем наличие доступных весов и загружаем их
        checkpoint_name = os.path.join(resume_from_checkpoint, "pytorch_model.bin")  # Полный чекпоинт
        if not os.path.exists(checkpoint_name):
            checkpoint_name = os.path.join(resume_from_checkpoint, "adapter_model.bin")
            resume_from_checkpoint = False  # чтобы тренер не пытался загрузить его состояние
        if os.path.exists(checkpoint_name):
            print(f"Продолжаем обучение с {checkpoint_name}")
            adapters_weights = torch.load(checkpoint_name)
            set_peft_model_state_dict(model, adapters_weights)
        else:
            print(f"Чекпоинт {checkpoint_name} не найден")

    model.print_trainable_parameters()  #Отображаем % обучаемых параметров

In [ ]:
# Обучение gpt2
def train_gpt2(
    dataset: dataset,
    base_model: str = "gpt2-medium",
    max_length: int = 512,
    num_epochs: int = 8,
    batch_size: int = 8,
    learning_rate: float = 2e-5,
    resume_from_checkpoint: str = None,
    micro_batch_size: int = 4,
    lora_r: int = 8,
    lora_alpha: int = 16,
    lora_dropout: float = 0.05,
    train_on_inputs: bool = True,
    add_eos_token: bool = False,
    val_set_size: int = 0
):
    tokenizer = GPT2Tokenizer.from_pretrained(base_model)
    tokenizer.pad_token = tokenizer.eos_token

    model = GPT2LMHeadModel.from_pretrained(base_model)

    model = prepare_model_for_int8_training(model)

    config = LoraConfig(
        r=lora_r,
        lora_alpha=lora_alpha,
        lora_dropout=lora_dropout,
        bias="none",
        task_type="CAUSAL_LM",
    )
    model = get_peft_model(model, config)

    resume_training(model, resume_from_checkpoint)

    dataset = dataset
    train_dataset, val_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

    training_args = TrainingArguments(
        output_dir="/content/drive/MyDrive/МФТИ/nlp2/chatbot/generative/gpt2results",
        per_device_train_batch_size=micro_batch_size,
        gradient_accumulation_steps=batch_size // micro_batch_size,
        warmup_steps=100,
        evaluation_strategy="steps" if val_set_size > 0 else "no",
        save_strategy="steps",
        eval_steps=200 if val_set_size > 0 else None,
        save_steps=200,
        save_total_limit=3,
        num_train_epochs=num_epochs,
        load_best_model_at_end=True if val_set_size > 0 else False,
    )

    os.makedirs(training_args.output_dir, exist_ok=True)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        data_collator=DataCollatorForSeq2Seq(
            tokenizer, return_tensors="pt", padding=True
        ),
    )

    model.config.use_cache = False

    old_state_dict = model.state_dict()

    model.state_dict = (
        lambda self, *_, **__: get_peft_model_state_dict(
            self, old_state_dict
        )
    ).__get__(model, type(model))

    trainer.train(resume_from_checkpoint=resume_from_checkpoint)

    model.save_pretrained(training_args.output_dir)

    print("Training complete.")

In [ ]:
train_gpt2(dataset)

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:145: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/layer.py:861: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


trainable params: 786,432 || all params: 355,609,600 || trainable%: 0.2211503851414585


Step,Training Loss
500,1.710700
1000,0.621000
1500,0.595200
2000,0.578000
2500,0.570400
3000,0.564600
3500,0.557200
4000,0.558100


Training complete.
